In [ ]:
import numpy as np
import soundfile
import torch
from tqdm import tqdm
import torch.utils.data
import h5py

from hppnet.constants import *
from hppnet.midi import parse_midi
import os

midi to tsv

In [ ]:
path = 'F:/dataset'
fns = [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.mid')]
for fn in tqdm(fns):
    try:
        midi_path = fn
        tsv_filename = fn[:-3]+'tsv'
        midi = parse_midi(midi_path)
        np.savetxt(tsv_filename, midi, fmt='%.6f', delimiter='\t', header='onset,offset,note,velocity')
    except:
        print(fn)

pack tsv,wav to h5

In [ ]:
path = 'F:/dataset'
fns = [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.mid')]
for fn in tqdm(fns):
    try:
        midi_path = fn
        tsv_path = fn[:-3]+'tsv'
        h5_path = fn[:-3]+'h5'
        audio_path = fn[:-3]+'wav'

        with h5py.File(h5_path, mode='w') as h5:                
            audio, sr = soundfile.read(audio_path, dtype='int16')
            # audio, sr = librosa.load(audio_path, sr = SAMPLE_RATE, mono=True)
            assert sr == SAMPLE_RATE

            audio = torch.ShortTensor(audio)
            audio_length = len(audio)

            n_keys = MAX_MIDI - MIN_MIDI + 1
            n_steps = (audio_length - 1) // HOP_LENGTH + 1

            label = torch.zeros(n_steps, n_keys, dtype=torch.uint8)
            velocity = torch.zeros(n_steps, n_keys, dtype=torch.uint8)


            if(len(tsv_path) > 0):
                tsv_path = tsv_path
                # print(tsv_path)
                midi = np.loadtxt(tsv_path, delimiter='\t', skiprows=1)
                if len(midi.shape) == 1:
                    midi = np.expand_dims(midi,0)

                for onset, offset, note, vel in midi:
                    left = int(round(onset * SAMPLE_RATE / HOP_LENGTH))
                    onset_right = min(n_steps, left + HOPS_IN_ONSET)
                    frame_right = int(round(offset * SAMPLE_RATE / HOP_LENGTH))
                    frame_right = min(n_steps, frame_right)
                    offset_right = min(n_steps, frame_right + HOPS_IN_OFFSET)

                    f = int(note) - MIN_MIDI
                    label[left:onset_right, f] = 3
                    label[onset_right:frame_right, f] = 2
                    label[frame_right:offset_right, f] = 1
                    velocity[left:frame_right, f] = vel

            # data = dict(path=audio_path, audio=audio, label=label, velocity=velocity)
            h5['path'] = audio_path
            h5['audio'] = audio.numpy()
            h5['label'] = label.numpy()
            h5['velocity'] = velocity.numpy()
            # torch.save(data, saved_data_path)
    except:
        print(fn)